In [1]:
#Decision Trees and Random Forests
#Sukhada Sheth
#Data Mining - Assignment 3

In [2]:

import pandas as pd

data_filename = "basketball.csv"
dataset = pd.read_csv(data_filename)

dataset.head() #print top 5 rows

,Date,Start (ET),Visitor/Neutral,PTS,Home/Neutral,PTS.1,Unnamed: 6,Unnamed: 7,Attend.,Notes
0,Tue Oct 27 2015,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
1,Tue Oct 27 2015,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
2,Tue Oct 27 2015,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,Wed Oct 28 2015,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN
4,Wed Oct 28 2015,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN


In [3]:
#format the date column
dataset = pd.read_csv(data_filename, parse_dates=["Date"])
#label the columns of the dataset
dataset.columns = ["Date", "Start (ET)", "Visitor Team", "VisitorPts", "Home Team", "HomePts", "OT?", "Score Type", "Attend.", "Notes"]

dataset.head() #print top 5 rows

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attend.,Notes
0,2015-10-27,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN
1,2015-10-27,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN
2,2015-10-27,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN
3,2015-10-28,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN
4,2015-10-28,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN


In [4]:
print(dataset.dtypes) #print datatype of each column of dataset

dataset["HomeWin"] = dataset["VisitorPts"] < dataset["HomePts"]  #add new feature/column depending on who has more points

dataset.head()

Date            datetime64[ns]
Start (ET)              object
Visitor Team            object
VisitorPts               int64
Home Team               object
HomePts                  int64
OT?                     object
Score Type              object
Attend.                  int64
Notes                   object
dtype: object


,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attend.,Notes,HomeWin
0,2015-10-27,8:00p,Detroit Pistons,106,Atlanta Hawks,94,Box Score,NaN,19187,NaN,False
1,2015-10-27,8:00p,Cleveland Cavaliers,95,Chicago Bulls,97,Box Score,NaN,21957,NaN,True
2,2015-10-27,10:30p,New Orleans Pelicans,95,Golden State Warriors,111,Box Score,NaN,19596,NaN,True
3,2015-10-28,7:30p,Philadelphia 76ers,95,Boston Celtics,112,Box Score,NaN,18624,NaN,True
4,2015-10-28,7:30p,Chicago Bulls,115,Brooklyn Nets,100,Box Score,NaN,17732,NaN,False


In [5]:
y_true = dataset["HomeWin"].values #create list of class for homewinner
print(type(y_true))

dataset["HomeWin"].mean()  #mean of homewinner chances

from collections import defaultdict
won_last = defaultdict(int)

#create two new datsets
dataset["HomeLastWin"] = 0  
dataset["VisitorLastWin"] = 0

<class 'numpy.ndarray'>


In [6]:
#for each row , update the winner of previous game 

for index, row in dataset.iterrows():
    home_team = row["Home Team"]
    visitor_team = row["Visitor Team"]
    row["HomeLastWin"] = won_last[home_team]
    dataset.at[index, "HomeLastWin"] = won_last[home_team]
    dataset.at[index, "VisitorLastWin"] =  won_last[visitor_team]
#also, create two more class list for last time winners    
    won_last[home_team] = int(row["HomeWin"])
    won_last[visitor_team] = 1 - int(row["HomeWin"])
    
dataset.tail(6)

,Date,Start (ET),Visitor Team,VisitorPts,Home Team,HomePts,OT?,Score Type,Attend.,Notes,HomeWin,HomeLastWin,VisitorLastWin
1310,2016-06-05,8:00p,Cleveland Cavaliers,77,Golden State Warriors,110,Box Score,NaN,19596,NaN,True,1,0
1311,2016-06-08,9:00p,Golden State Warriors,90,Cleveland Cavaliers,120,Box Score,NaN,20562,NaN,True,0,1
1312,2016-06-10,9:00p,Golden State Warriors,108,Cleveland Cavaliers,97,Box Score,NaN,20562,NaN,False,1,0
1313,2016-06-13,9:00p,Cleveland Cavaliers,112,Golden State Warriors,97,Box Score,NaN,19596,NaN,False,1,0
1314,2016-06-16,9:00p,Golden State Warriors,101,Cleveland Cavaliers,115,Box Score,NaN,20562,NaN,True,1,0
1315,2016-06-19,8:00p,Cleveland Cavaliers,93,Golden State Warriors,89,Box Score,NaN,19596,NaN,False,0,1


In [7]:
dataset.iloc[1000:1005]

#create numpy array 
X_previouswins = dataset[["HomeLastWin", "VisitorLastWin"]].values
print(type(X_previouswins))

<class 'numpy.ndarray'>


In [ ]:
####################decision tree application starts ##################

from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=14)

from sklearn.model_selection import cross_val_score
import numpy as np
